# [Clustering4Ever](https://github.com/Clustering4Ever/Clustering4Ever) on [SparkNotebook](http://spark-notebook.io/) by [LIPN](https://lipn.univ-paris13.fr/) [A3](https://lipn.univ-paris13.fr/accueil/equipe/a3/) team

# Scala _K_-Means

In [ ]:
import smile.plot._
import scala.io.Source
import scala.collection.{immutable, mutable, parallel}
import clustering4ever.scala.clustering.kmeans.{KMeans, KMeansModel}
import clustering4ever.math.distances.scalar.{Euclidean, Cosine, Minkowski}
import clustering4ever.spark.indexes.ExternalIndexes
import clustering4ever.scala.indexes.{ExternalIndexes => ScalaExternalIndexes}
import clustering4ever.scala.indexes.NmiNormalizationNature._
import clustering4ever.util.ScalaImplicits._
import clustering4ever.scala.clusterizables.RealClusterizable
import clustering4ever.scala.vectorizables.RealVector

import smile.plot._
import scala.io.Source
import scala.collection.{immutable, mutable, parallel}
import clustering4ever.scala.clustering.kmeans.{KMeans, KMeansModel}
import clustering4ever.math.distances.scalar.{Euclidean, Cosine, Minkowski}
import clustering4ever.spark.indexes.ExternalIndexes
import clustering4ever.scala.indexes.{ExternalIndexes=>ScalaExternalIndexes}
import clustering4ever.scala.indexes.NmiNormalizationNature._
import clustering4ever.util.ScalaImplicits._
import clustering4ever.scala.clusterizables.RealClusterizable
import clustering4ever.scala.vectorizables.RealVector


## Download dataset Aggregation

In [ ]:
:sh wget -P /tmp/ http://www.clustering4ever.org/Datasets/Aggregation/aggregation.csv
:sh wget -P /tmp/ http://www.clustering4ever.org/Datasets/Aggregation/labels

--2018-09-02 23:20:10--  http://www.clustering4ever.org/Datasets/Aggregation/aggregation.csv
Resolving www.clustering4ever.org (www.clustering4ever.org)... 62.210.16.62
Connecting to www.clustering4ever.org (www.clustering4ever.org)|62.210.16.62|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 8063 (7.9K) [text/csv]
Saving to: ‘/tmp/aggregation.csv.10’

     0K .......                                               100%  799K=0.01s

2018-09-02 23:20:20 (799 KB/s) - ‘/tmp/aggregation.csv.10’ saved [8063/8063]

:sh: Scheme missing.
--2018-09-02 23:20:20--  http://wget/
Resolving wget (wget)... failed: Name or service not known.
wget: unable to resolve host address ‘wget’
--2018-09-02 23:20:20--  http://www.clustering4ever.org/Datasets/Aggregation/labels
Reusing existing connection to www.clustering4ever.org:80.
HTTP request sent, awaiting response... 200 OK
Length: 1576 (1.5K)
Saving to: ‘/tmp/labels.11’

     0K .                                                   

## Import data as Array[Array[Double]]

In [ ]:
val datasetSize = 1000
val dim = 20

val path = "/tmp/aggregation.csv"

val rawAggregationData = Source.fromFile(path).getLines.map( x => mutable.ArrayBuffer(x.split(",").map(_.toDouble):_*) ).toBuffer.zipWithIndex

val randomRawdata = mutable.ArrayBuffer.fill(datasetSize)(mutable.ArrayBuffer.fill(dim)(scala.util.Random.nextDouble)).zipWithIndex

val data = rawAggregationData.map{ case (v, id) => new RealClusterizable(id.toLong, new RealVector(v)) }

val labelsPath = "/tmp/labels"

datasetSize: Int = 1000
dim: Int = 20
highDim: Boolean = true
path: String = /tmp/aggregation.csv
rawAggregationData: scala.collection.mutable.Buffer[(scala.collection.mutable.ArrayBuffer[Double], Int)] = ArrayBuffer((ArrayBuffer(15.55, 28.65),0), (ArrayBuffer(14.9, 27.55),1), (ArrayBuffer(14.45, 28.35),2), (ArrayBuffer(14.15, 28.8),3), (ArrayBuffer(13.75, 28.05),4), (ArrayBuffer(13.35, 28.45),5), (ArrayBuffer(13.0, 29.15),6), (ArrayBuffer(13.45, 27.5),7), (ArrayBuffer(13.6, 26.5),8), (ArrayBuffer(12.8, 27.35),9), (ArrayBuffer(12.4, 27.85),10), (ArrayBuffer(12.3, 28.4),11), (ArrayBuffer(12.2, 28.65),12), (ArrayBuffer(13.4, 25.1),13), (ArrayBuffer(12.95, 25.95),14), (ArrayBuffer(12.9, 26.5),15), (ArrayBuffer(11.85, 27.0),16), (ArrayBuffer(11.35, 28.0),17), (ArrayBuffer(11.15, 28.7),18), ...

## Parameters 

In [ ]:
val k = 7
val iterMax = 40
val epsilon = 0.5
// True defines the true Euclidean with square root, set it to false to avoid the square
// val metric0 = new Euclidean[Array[Double]](true)
val metric1 = new Euclidean[mutable.ArrayBuffer[Double]](true)

// Minkowski(p) where p is the Minkowski parameter
val metric2 = new Minkowski[mutable.ArrayBuffer[Double]](2)

k: Int = 7
iterMax: Int = 40
epsilon: Double = 0.5
metric1: clustering4ever.math.distances.scalar.Euclidean[scala.collection.mutable.ArrayBuffer[Double]] = Euclidean with root applied
metric2: clustering4ever.math.distances.scalar.Minkowski[scala.collection.mutable.ArrayBuffer[Double]] = clustering4ever.math.distances.scalar.Minkowski@653843b6


## Run the algorithm

In [ ]:
val t1 = System.nanoTime

val model = KMeans.run[Long,
              mutable.ArrayBuffer[Double],
              mutable.ArrayBuffer[Double],
              RealClusterizable[Long, mutable.ArrayBuffer[Double], mutable.ArrayBuffer[Double]],
              Euclidean[mutable.ArrayBuffer[Double]]
              ](data, k, epsilon, iterMax, metric1)

val t2 = System.nanoTime

(t2 - t1) / 1000000000D

t1: Long = 44758892917081
model: clustering4ever.scala.clustering.kmeans.KMeansModel[Long,scala.collection.mutable.ArrayBuffer[Double],scala.collection.mutable.ArrayBuffer[Double],clustering4ever.scala.clusterizables.RealClusterizable[Long,scala.collection.mutable.ArrayBuffer[Double],scala.collection.mutable.ArrayBuffer[Double]],clustering4ever.math.distances.scalar.Euclidean[scala.collection.mutable.ArrayBuffer[Double]]] = clustering4ever.scala.clustering.kmeans.KMeansModel@5e24fbbd
t2: Long = 44758902785310
res30: Double = 0.009868229


0.009868229

In [ ]:
val clusterized11 = rawAggregationData.map{ case (v, id) => (model.centerPredict(v), v) }
val clusterized12 = model.centerPredict(rawAggregationData.map(_._1))

val clusterized21 = data.map( clustbl => (model.centerPredict(clustbl.vector), clustbl) )
val clusterized22 = model.centerPredict(data)

clusterized11: scala.collection.mutable.Buffer[(model.ClusterID, scala.collection.mutable.ArrayBuffer[Double])] = ArrayBuffer((6,ArrayBuffer(15.55, 28.65)), (6,ArrayBuffer(14.9, 27.55)), (6,ArrayBuffer(14.45, 28.35)), (6,ArrayBuffer(14.15, 28.8)), (6,ArrayBuffer(13.75, 28.05)), (6,ArrayBuffer(13.35, 28.45)), (6,ArrayBuffer(13.0, 29.15)), (6,ArrayBuffer(13.45, 27.5)), (6,ArrayBuffer(13.6, 26.5)), (6,ArrayBuffer(12.8, 27.35)), (6,ArrayBuffer(12.4, 27.85)), (6,ArrayBuffer(12.3, 28.4)), (6,ArrayBuffer(12.2, 28.65)), (6,ArrayBuffer(13.4, 25.1)), (6,ArrayBuffer(12.95, 25.95)), (6,ArrayBuffer(12.9, 26.5)), (6,ArrayBuffer(11.85, 27.0)), (6,ArrayBuffer(11.35, 28.0)), (6,ArrayBuffer(11.15, 28.7)), (6,ArrayBuffer(11.25, 27.4)), (6,ArrayBuffer(10.75, 27.7)), (6,ArrayBuffer(10.5, 28.35)), (6,ArrayBu...

## Plot clustering results

In [ ]:
val rawData = clusterized22.map( rc => rc.vector.toArray ).toArray
val labels = clusterized22.map( rc => rc.clusterID ).toArray
plot(rawData, labels, '*', Palette.COLORS)

The cell was cancelled.


## Inspect performance metrics

In [ ]:
val trueLabels = Source.fromFile(labelsPath).getLines.map(_.toInt).toVector

val (_, trueLabelsFrom0) = ScalaExternalIndexes.prepareLabels(trueLabels)

val predicts = clusterized22.map(_.clusterID).toVector

val trueAndPredictRDD2 = sc.parallelize(trueLabelsFrom0.zip(predicts).seq)

val nmi = ScalaExternalIndexes.nmi(trueLabelsFrom0, predicts, SQRT)
val nmi2 = ExternalIndexes.nmi(sc, trueAndPredictRDD2, SQRT)

(nmi, nmi2)

trueLabels: Vector[Int] = Vector(2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,...

(0.8377117570406222,0.8377117570406222)